# Genetic algorithm for Web based game

<i><strong style="color:red;">N.B. When running this program a local server will start on <a href="http://127.0.0.1:5000" target="_blank">127.0.0.1:5000</a> - Open in another tab to see</strong></i>

In [1]:
import numpy as np
import json
import tensorflow as tf
from numpy.random import randint
import random
from random import random as rnd
from flask import Flask, render_template, request, jsonify
from flask_cors import CORS
from keras.layers import Dense, Concatenate,concatenate,LeakyReLU, Softmax
from keras.models import Sequential, Model
from keras.optimizers import Adam
import math
import logging
from sklearn.preprocessing import normalize

Using TensorFlow backend.


## Init app

In [2]:
# Define App
app = Flask(__name__)
CORS(app)

# Hide logging
log = logging.getLogger('werkzeug')
log.setLevel(logging.ERROR) # Comment out to see server requests

## Variable definitions

In [3]:
ready = False
input_size = 2
output_size = 1
population = []
max_fitness = 0
avg_score = 0
mating_pool = []
neurons = 4
generation = 0
processed_count = 0
leaky = True
shuffle_mating_pool = False

# Error fix
graph = tf.get_default_graph()

## Hyperparameters

In [4]:
number_of_individuals = 20
mutation_rate = 0.01
breeding_rate = 0
learning_rate = 0.098
leaky_alpha =0.3

## Class definitions

In [5]:
class Bird:
    def __init__(self):
        self.brain = self.create_brain()
        self.score = 0
        self.fitness = 0
        self.prediction = None
        self.alive = 1

    def create_brain(self):
        model = Sequential()    
        if leaky:
            model.add(Dense(input_size, input_dim=input_size))
            model.add(LeakyReLU(alpha=leaky_alpha))
            model.add(Dense(neurons))
            model.add(LeakyReLU(alpha=leaky_alpha))
        else:
            model.add(Dense(input_size, input_dim=input_size, activation='relu'))
            model.add(Dense(neurons, activation='relu'))
        model.add(Dense(output_size))
        #model.add(Softmax())
        model.compile(loss='mse', optimizer=Adam(lr=learning_rate), metrics=["accuracy"])
        model._make_predict_function()
        return model
    
    def update_brain(self, layer_0,layer_1,layer_2):
        global leaky
        if leaky:
            self.brain.layers[0].set_weights([np.array(layer_0),np.zeros(input_size)])
            self.brain.layers[2].set_weights([np.array(layer_1),np.zeros(neurons)])
            self.brain.layers[4].set_weights([np.array(layer_2),np.zeros(output_size)])
        else:
            self.brain.layers[0].set_weights([np.array(layer_0),np.zeros(input_size)])
            self.brain.layers[1].set_weights([np.array(layer_1),np.zeros(neurons)])
            self.brain.layers[2].set_weights([np.array(layer_2),np.zeros(output_size)])
    
    def predict(self, pipe_states):
        if self.alive == 0:
            return self.prediction
        return_val = []
        inputs = [pipe_states[0], pipe_states[1]]
        reshaped = np.array([inputs])
        prediction = self.brain.predict(reshaped)
        self.prediction = prediction.tolist()
        return self.prediction
    
    def reset(self):
        self.score = 0
        self.fitness = 0
        self.alive = 1
        self.prediction = None
    
    def set_status(self, status):
        self.alive = status
    
    def set_score(self, score):
        self.score = score

## Method definitions

In [6]:
def create_first_generation(callback):
    global ready, generation
    for i in range(number_of_individuals):
        bird = Bird()
        population.append(bird)
    
    generation = generation+1
    ready = True
    callback()

In [7]:
def bird_predictions(bird_states, pipe_states):
    if not bird_states or not pipe_states:
        return
    
    bird_predictions_array = []
    for i in range(len(population)):
        population[i].set_score(bird_states[i][2])
        population[i].set_status(bird_states[i][0])
        temp_predict = population[i].predict(pipe_states)
        bird_predictions_array.append(temp_predict)
    return bird_predictions_array

In [8]:
def calculate_fitness():
    global max_fitness, avg_score, number_of_individuals
    sum = 0
    for i in range(len(population)):
        sum = sum+population[i].score

    avg_score = sum/number_of_individuals
    max_fitness = 0
    for i in range(len(population)):
        population[i].fitness = population[i].score/sum
        if population[i].fitness > max_fitness:
            max_fitness = population[i].fitness

In [9]:
def natural_selection(filter = True): #Mating pool
    global max_fitness, mating_pool
    mating_pool = []
    for i in range(len(population)):
        n = math.floor(population[i].fitness*100)
        for j in range(n):
            mating_pool.append(i)
            
    if shuffle_mating_pool:
        random.shuffle(mating_pool)

In [10]:
def get_dense_layers(layers):
    global leaky
    if leaky:
        return (layers[0].get_weights()[0], 
                layers[2].get_weights()[0], 
                layers[4].get_weights()[0])
    else:
        return (layers[0].get_weights()[0], 
                layers[1].get_weights()[0], 
                layers[2].get_weights()[0]) 

In [11]:
def select_weights(weights_1, weights_2):
    select = []
    weight_pool = []
    size = 0
    for i in range(len(weights_1)):
        size = len(weights_1[i])
        for j in range(size):
            weight_pool.append(weights_1[i][j])
            weight_pool.append(weights_2[i][j])
    random.shuffle(weight_pool)
    for i in range(len(weights_1)):
        select.append(random.sample(weight_pool, size))
    return select

In [12]:
def ordered_select_weights(weights_1, weights_2):
    select = []
    N = 0
    for i in range(len(weights_1)):
        N = len(weights_1[i])
        from_which_list = np.random.randint(2, size=N).astype(np.bool)
        select.append(np.choose(from_which_list, [weights_1[i], weights_2[i]]))

    return select

In [13]:
def multiply_weights(weights_1, weights_2):
    multiplied = []
    for i in range(len(weights_1)):
        temp = normalize( weights_1[i].reshape(-1, 1) * weights_2[i].reshape(1, -1) )
        multiplied.append( np.random.choice( temp[0] , 4).tolist() )
    return multiplied

In [14]:
def dot_weights(weights_1, weights_2):
    return np.reshape(normalize(np.dot( np.array(weights_1).reshape(-1, 1),np.array(weights_2).reshape(1, -1) ) )[0], weights_1.shape )

In [15]:
def crossover(father, mother, orig):
    (father_0,father_1,father_2) = get_dense_layers(father.brain.layers)
    (mother_0,mother_1,mother_2) = get_dense_layers(mother.brain.layers)
    (orig_0,orig_1,orig_2) = get_dense_layers(orig.brain.layers)

    #child_0 = dot_weights(father_0, mother_0)
    #child_1 = dot_weights(father_1, mother_1)
    #child_2 = dot_weights(father_2, mother_2)
    
    child_0 = ordered_select_weights(father_0,mother_0)
    child_1 = ordered_select_weights(father_1,mother_1)
    child_2 = ordered_select_weights(father_2,mother_2)
    
    return (child_0,child_1,child_2)

In [16]:
def breed_population():
    global mating_pool, ready, generation, processed_count
    for i in range(len(population)):
        # Get two random indecies from the mating pool 
        a = mating_pool[math.floor(np.random.choice(len(mating_pool)))]
        b = mating_pool[math.floor(np.random.choice(len(mating_pool)))]
        (layer_0, layer_1, layer_2) = crossover(population[a], population[b], population[i])
        layer_0 = mutate(layer_0)
        layer_1 = mutate(layer_1)
        layer_2 = mutate(layer_2)
        population[i].update_brain(layer_0,layer_1,layer_2)
        population[i].reset()
        processed_count = processed_count+1
    generation = generation+1
    server_ready()

In [17]:
def mutate(individual_weights):
    global mutation_rate
    '''
    length = len(individual_weights[0])
    for swapped in range(length):
        if(random.random() < mutation_rate):
            swap_with = int(random.random() * length)
            weight1 = individual_weights[0][swapped]
            weight2 = individual_weights[0][swap_with]
            individual_weights[0][swapped] = weight1
            individual_weights[0][swap_with] = weight2

    '''
    for i in range(len(individual_weights)):
        if(random.random() < mutation_rate):
            individual_weights[i][random.randint(0,len(individual_weights[i])-1)]=random.uniform(-1, 1)
            #random.shuffle(individual_weights[i])
            
    return individual_weights

## Routes for server/client communication

In [18]:
# Create route for requesting the individuals NN brain
@app.route("/predictions")
def get_predictions():
    global ready
    if not ready:
        return "server busy"
    parsed_bird_states = (json.loads(request.args.get('bird_states')))
    parsed_pipe_states = (json.loads(request.args.get('pipe_states')))
    if len(parsed_bird_states) == 0 or len(parsed_pipe_states) == 0:
        return "No birds"
    global graph
    with graph.as_default():
        try:
            return jsonify(bird_predictions(parsed_bird_states, parsed_pipe_states))
        except:
            print("Warning")
            return "Warning"

In [19]:
@app.route("/is_ready")
def is_ready():
    global ready
    return str(ready)

In [20]:
@app.route("/get_next_generation")
def get_next_generation():
    server_busy()
    calculate_fitness()
    natural_selection()
    debug_this = False
    if debug_this == True:
        breed_population()
        return "breed_debug"
    else:
        global graph
        with graph.as_default():
            try:
                breed_population()
                return "breeding population"
            except:
                print("breed_population() exception")
                return "breed_population() exception"

In [21]:
# Create route for requesting the number of individuals
@app.route("/get_count")
def get_count():
    return str(number_of_individuals)

In [22]:
@app.route("/get_info")
def get_info():
    global generation,max_fitness,mating_pool,avg_score,number_of_individuals
    info = [
        generation,
        number_of_individuals,
        max_fitness,
        len(mating_pool),
        avg_score
    ]
    return jsonify(info)

In [23]:
# Create route for index file
@app.route("/")
def index():
    return render_template("index.html")

## Run program

In [24]:
def server_ready():
    global ready, processed_count    
    if processed_count == number_of_individuals:
        ready = True
        processed_count = 0
    else:
        ready = False
    #print("server is ready")

In [25]:
def server_busy():
    global ready
    ready = False
    #print("server is busy")

In [26]:
def run_app():
    app.run(debug=False)

In [ ]:
# Run program
if __name__ == '__main__':
    # Start the app
    create_first_generation(run_app)

Instructions for updating:
Colocations handled automatically by placer.
 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off
